# Proces Mining - fake dataset PowerBI dag

In [9]:
# Importeer packages
import pandas as pd
import numpy as np
import uuid
from random import normalvariate, choice
import random
import datetime 
from faker import Faker

## Classes en functies

In [10]:
# Maakt een random datetime aan de hand van een startdatetime en een interval
def random_date(start,interval):
    if interval == 0:
        return start
    else:
        delta = (np.random.poisson(1,1)+1)+normalvariate(interval, interval*0.1)
        current = start + datetime.timedelta(minutes = delta[0])
        return current

In [11]:
# Elke stap is een class, met:
# - Een naam
# - Mogelijke volgende stappen (in lijst)
# - Kansen op die volgende stappen (in lijst)
# - Tijd binnen huidige stap (om verschil tussen start en eind te bepalen)
# - Tijd tussen huidige stap en volgende stap (verschil tussen eind huidige stap en begin volgende, in lijst)
# - Tijd waarbinnen de start en eindtijden moeten vallen, voor nu altijd tussen 8.00-18.00

class step():
    instances = []
    def __init__(self,name, next_step, chance_next_step, time_step,time_to_next_step, possible_interval =[8,18]):
        self.__class__.instances.append(self)
        self.name = name
        self.next_step = next_step
        self.chance_next_step = chance_next_step
        self.time_step = time_step
        self.time_to_next_step = time_to_next_step
        self.possible_interval = possible_interval

In [12]:
# Elk datapunt (een case van begin tot eind in het proces) is een class. 
class datapoint():
    # Starteigenschappen
    def __init__(self): # Begin met eerste stap van datapunt
        self.userID = uuid.uuid4()
        self.controller = random.choice(['Controller1','Controller2','Controller3','Controller4','Controller5'])
        self.bo = random.choice(['BO1','BO2','BO3','BO4','BO5','BO6','BO7','BO8','BO9','BO10'])
        self.afdeling = random.choice(['Sales','Marketing','Inkoop'])
        self.currentstep = START
        self.stepmin2 = "geen"
        self.stepmin1 = "geen"
        
        
        # Startdatum random tussen 1-7-2018 en 1-10-2018, als niet in timewindow of in weekend, voeg telkens een uur toe
        fake = Faker()
        self.start = fake.date_time_between_dates(datetime_start=datetime.datetime(2018,1,1), datetime_end=datetime.datetime(2019,1,1))
        while self.start.hour < self.currentstep.possible_interval[0] or self.start.hour >= self.currentstep.possible_interval[1] or self.start.weekday()>=5:
            self.start = random_date(self.start,60) 
            
        # Einddatum. Als niet in timewindow of in weekend, voeg telkens een uur toe.
        self.end = random_date(self.start, self.currentstep.time_step)
        while self.end.hour < self.currentstep.possible_interval[0] or self.end.hour >= self.currentstep.possible_interval[1] or self.end.weekday()>=5:
            self.end = random_date(self.end,60) 
            
    # Functie die de case van begin tot eind in proces maakt    
    def create_datapoint(self,df):
        # Append eerste rij
        df = df.append({'ID':self.userID, 'Step':self.currentstep.name, 'Start':self.start, 'End':self.end,
                        'Controller':self.controller,'BusinessOwner':self.bo,'Afdeling':self.afdeling}, ignore_index = True)     
        
        # Herhaal totdat de stap 'DONE' is bereikt
        while self.currentstep.name != 'DONE':
            rand = random.random()
            sumchance = 0
                                    
            # Vergelijk kansen met random nummer om volgende stap te bepalen
            for i in range(len(self.currentstep.chance_next_step)):
                sumchance += self.currentstep.chance_next_step[i]
                
                # Bij een bepaald patroon van stappen wordt de next step altijd CPR: sumchance 1
                if self.stepmin2 == 'START' and \
                    self.stepmin1=='RIN' and \
                    self.currentstep.name =='INR' and \
                    self.currentstep.next_step[i] =='CPR':
                    sumchance = 1
                     
                if rand < sumchance:
                    # Loop through isntances to find correct one
                    
                    for instance in step.instances:
                        if instance.name == self.currentstep.next_step[i]:
                            
                            # Update alle variabelen van nieuwe stap
                            self.start = random_date(self.end, self.currentstep.time_to_next_step[i])
                            while self.start.hour < self.currentstep.possible_interval[0] or self.start.hour >= self.currentstep.possible_interval[1] or self.start.weekday()>=5:
                                self.start = random_date(self.start,60)
                            self.stepmin2 = self.stepmin1
                            self.stepmin1 = self.currentstep.name
                            self.currentstep = instance 
                            self.end = random_date(self.start, self.currentstep.time_step)
                            while self.end.hour < self.currentstep.possible_interval[0] or self.end.hour >= self.currentstep.possible_interval[1] or self.end.weekday()>=5:
                                self.end = random_date(self.end,60)
                                
                            # Als niet DONE, dan voeg toe aan df
                            #if self.currentstep.name != 'DONE':
                            df = df.append({'ID':self.userID, 'Step':self.currentstep.name, 'Start':self.start, 'End':self.end,
                                           'Controller':self.controller,'BusinessOwner':self.bo,'Afdeling':self.afdeling}, ignore_index = True)
                            
                            break
                    break

        return df

In [20]:
# Functie die de hele dataset, tot aan AantalN maakt
def create_dataset(AantalN):
    df = pd.DataFrame(data={'ID':[], 'Step':[], 'Start':[], 'End':[], 'Controller':[], 'BusinessOwner':[], 'Afdeling':[]})
    while len(df) < AantalN:
        print(len(df))
        df = datapoint().create_datapoint(df)
    return df 

## Steps en dataset

In [21]:
steps_dict = {
    'CPR' : '1. Create/update purchase requisition',
    'ABC' : '2. Approval by controller',
    'ABO' : '3. Approval by business owner',
    'ABD' : '4. Approval by business director >50K',
    'CPO' : '5. Create purchase order',
    'SPO' : '6. Send purchase order to supplier',
    'CGR' : '7. Create goods receipt',
    'UGR' : '8. Update goods receipt',
    'RIN' : '9. Receive invoice',
    'INR' : '10. Invoice reconciliation',
    'IOK' : '11. Issue OK to pay',
    'CIR' : '12. Create invoice receipt',
    'EAP' : '13. Execute accounts payable'
}

In [28]:
# Het aanmaken van de verschillende stappen
# Instantiate new step : name, next steps, chances next steps, time during step, time to next step, possible times (optional)
week_min = 7*24*60
START = step('START', ['CPR', 'RIN'],[0.8,0.2], 0, [0,0])
CPR = step('CPR', ['ABC'],[1], 10, [8*60])
ABC = step('ABC', ['CPR', 'ABO'],[0.25,0.75], 5, [4*24*60,2*24*60])
ABO = step('ABO',['ABD', 'CPO', 'CPR'], [0.3,0.68,0.02], 2,[2*24*60, 5*24*60,4*24*60])
ABD = step('ABD',['CPR','CPO'],[0.005,0.995],60, [week_min,0])
CPO = step('CPO',['SPO', 'DONE'],[0.95, 0.05], 0,[0,0])
SPO = step('SPO',['CGR'],[1], 0,[2*week_min])
CGR = step('CGR',['UGR'],[1], 5,[2*24*60])
UGR = step('UGR',['RIN'],[1], 5,[0])
RIN = step('RIN',['INR'],[1], 0,[0])
INR = step('INR',['CPR', 'IOK'],[0.05,0.95], 0,[3*24*60,0])
IOK = step('IOK',['UGR', 'CIR'],[0.15,0.85], 0,[4*24*60,0])
CIR = step('CIR',['EAP'],[1], 0,[0])
EAP = step('EAP',['RIN', 'CGR', 'DONE'],[0.06,0.18,0.76], 0,[3*24*60,4*24*60,0])
DONE = step('DONE',[],[],0,[0])

In [33]:
# Het aanmaken van de dataset met lengte AantalN
dataset = create_dataset(100000)

0
21
37
51
65
83
104
125
151
165
196
211
247
263
281
298
323
362
384
398
405
419
434
448
464
492
518
538
554
569
604
625
642
648
671
679
694
710
727
744
774
794
810
828
845
859
878
892
907
921
950
964
985
999
1007
1037
1062
1068
1082
1098
1120
1142
1162
1188
1202
1217
1245
1261
1276
1290
1306
1312
1329
1344
1362
1393
1409
1439
1455
1476
1491
1507
1525
1543
1564
1588
1615
1629
1646
1660
1674
1699
1713
1735
1751
1765
1782
1788
1802
1825
1839
1862
1878
1896
1910
1924
1939
1957
1988
2002
2027
2041
2055
2080
2094
2114
2136
2152
2178
2192
2209
2231
2247
2265
2279
2285
2299
2315
2333
2352
2366
2391
2405
2427
2472
2486
2502
2525
2539
2553
2585
2606
2616
2623
2641
2657
2679
2697
2712
2734
2748
2762
2778
2792
2810
2826
2844
2860
2875
2892
2939
2967
3001
3026
3042
3062
3077
3098
3116
3138
3158
3173
3187
3204
3221
3237
3255
3285
3304
3324
3338
3354
3376
3390
3406
3422
3447
3465
3479
3497
3511
3532
3556
3572
3586
3596
3633
3678
3693
3721
3758
3780
3809
3823
3838
3859
3874
3890
3906
3934
3948
3966
3

27408
27426
27440
27462
27490
27505
27535
27549
27589
27603
27618
27634
27648
27673
27689
27705
27719
27725
27731
27750
27772
27791
27810
27855
27894
27908
27922
27944
27965
27980
27994
28008
28026
28040
28056
28079
28097
28112
28131
28180
28201
28232
28246
28264
28292
28309
28334
28348
28366
28380
28403
28419
28433
28447
28463
28490
28504
28521
28529
28575
28597
28614
28628
28650
28665
28680
28692
28706
28721
28745
28762
28801
28822
28854
28870
28888
28906
28921
28929
28944
28959
28976
28990
29011
29025
29053
29067
29093
29107
29122
29142
29156
29172
29186
29200
29214
29228
29242
29258
29281
29297
29311
29325
29356
29376
29382
29398
29418
29436
29450
29466
29482
29496
29512
29526
29540
29567
29589
29616
29630
29645
29662
29681
29696
29710
29724
29750
29767
29785
29800
29818
29840
29855
29872
29898
29925
29940
29954
29975
29983
30005
30026
30057
30074
30090
30104
30128
30147
30179
30205
30221
30239
30259
30274
30291
30320
30339
30355
30376
30390
30410
30429
30447
30463
30478
30506
3053

52614
52628
52651
52665
52689
52707
52721
52740
52755
52770
52791
52821
52835
52852
52869
52884
52891
52905
52921
52946
52971
52985
52999
53013
53037
53054
53071
53093
53116
53137
53160
53176
53200
53216
53233
53251
53269
53283
53313
53327
53341
53355
53376
53399
53414
53430
53444
53460
53480
53487
53505
53531
53550
53564
53582
53599
53613
53627
53652
53666
53712
53726
53771
53811
53827
53841
53855
53871
53891
53919
53945
53967
53983
54000
54020
54035
54054
54070
54086
54117
54138
54152
54172
54187
54216
54233
54254
54269
54301
54349
54374
54388
54409
54423
54438
54453
54468
54486
54509
54523
54545
54591
54619
54633
54647
54667
54681
54719
54735
54756
54770
54798
54824
54840
54866
54880
54896
54918
54933
54948
54965
54980
54995
55028
55042
55062
55082
55096
55110
55131
55165
55179
55207
55238
55253
55277
55291
55308
55329
55359
55387
55405
55427
55448
55462
55476
55490
55504
55518
55567
55573
55584
55598
55613
55627
55641
55659
55695
55711
55727
55754
55768
55785
55799
55820
55834
5587

78276
78299
78324
78345
78366
78381
78403
78417
78431
78453
78474
78490
78499
78513
78531
78563
78577
78583
78611
78626
78642
78660
78674
78688
78707
78728
78744
78784
78801
78817
78835
78849
78863
78879
78894
78900
78914
78929
78953
78967
78982
78999
79017
79033
79048
79054
79069
79083
79098
79112
79139
79154
79169
79189
79201
79218
79252
79266
79280
79295
79316
79334
79358
79386
79400
79414
79433
79462
79485
79509
79533
79558
79582
79596
79613
79627
79646
79660
79678
79693
79707
79721
79741
79757
79763
79779
79818
79838
79852
79866
79880
79899
79913
79930
79945
79961
79975
80000
80014
80030
80062
80076
80090
80106
80131
80145
80182
80209
80224
80244
80259
80273
80281
80295
80314
80329
80345
80381
80390
80424
80439
80454
80468
80482
80496
80512
80547
80562
80585
80601
80616
80630
80639
80660
80678
80697
80717
80747
80761
80783
80799
80818
80843
80857
80872
80892
80910
80931
80952
80967
80988
81002
81016
81037
81053
81067
81082
81096
81103
81117
81131
81145
81174
81190
81206
81225
8124

In [34]:
dataset['Step'] = dataset['Step'].replace(steps_dict)
dataset['ID'] = dataset['ID'].astype('category')
dataset['ID'] = dataset['ID'].cat.codes + 100000

In [35]:
dataset

,ID,Step,Start,End,Controller,BusinessOwner,Afdeling
0,102251,START,2018-01-08 08:15:40.684512,2018-01-08 08:15:40.684512,Controller2,BO2,Inkoop
1,102251,9. Receive invoice,2018-01-08 08:15:40.684512,2018-01-08 08:15:40.684512,Controller2,BO2,Inkoop
2,102251,10. Invoice reconciliation,2018-01-08 08:15:40.684512,2018-01-08 08:15:40.684512,Controller2,BO2,Inkoop
3,102251,1. Create/update purchase requisition,2018-01-15 17:39:44.838776,2018-01-15 17:52:05.797221,Controller2,BO2,Inkoop
4,102251,2. Approval by controller,2018-01-16 08:14:23.129770,2018-01-16 08:23:36.646765,Controller2,BO2,Inkoop
5,102251,3. Approval by business owner,2018-01-18 08:36:42.431986,2018-01-18 08:40:44.248295,Controller2,BO2,Inkoop
6,102251,5. Create purchase order,2018-01-24 08:22:26.067896,2018-01-24 08:22:26.067896,Controller2,BO2,Inkoop
7,102251,6. Send purchase order to supplier,2018-01-24 08:22:26.067896,2018-01-24 08:22:26.067896,Controller2,BO2,Inkoop
8,102251,7. Create goods receipt,2018-02-07 08:35:49.226825,2018-02-07 08:42:48.558376,Controller2,BO2,Inkoop
9,102251,8. Update goods receipt,2018-02-09 09:50:48.534644,2018-02-09 09:59:01.649895,Controller2,BO2,Inkoop


In [36]:
# Sla op als csv
dataset.to_csv("Proces_Mining_dataset_powerbidag_v5.csv")